In [3]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from mlforecast import MLForecast
from mlforecast.feature_engineering import transform_exog
from mlforecast.lag_transforms import RollingMean
from utilsforecast.evaluation import evaluate
from typing import List

# 2. Import data


In [2]:
# Read price data
price = pd.read_csv('Datasets\Phase 0 - Price.csv', na_values= np.nan)
price = price.melt(id_vars=["Client", "Warehouse","Product"], var_name='ds', value_name='Price')
price

,Client,Warehouse,Product,ds,Price
0,0,1,367,2020-07-06,10.900001
1,0,1,639,2020-07-06,NaN
2,0,1,655,2020-07-06,21.343332
3,0,1,1149,2020-07-06,11.480000
4,0,1,1485,2020-07-06,NaN
...,...,...,...,...,...
2559005,46,318,13485,2023-10-02,26.952000
2559006,46,318,13582,2023-10-02,24.679487
2559007,46,318,13691,2023-10-02,52.990000
2559008,46,318,13946,2023-10-02,30.990000


In [3]:
sales = pd.read_csv('Datasets\Phase 0 - Sales.csv', na_values= np.nan)
sales = sales.melt(id_vars=["Client", "Warehouse","Product"], var_name='ds', value_name='y')
sales

,Client,Warehouse,Product,ds,y
0,0,1,367,2020-07-06,7.0
1,0,1,639,2020-07-06,0.0
2,0,1,655,2020-07-06,21.0
3,0,1,1149,2020-07-06,7.0
4,0,1,1485,2020-07-06,0.0
...,...,...,...,...,...
2559005,46,318,13485,2023-10-02,80.0
2559006,46,318,13582,2023-10-02,39.0
2559007,46,318,13691,2023-10-02,1.0
2559008,46,318,13946,2023-10-02,3.0


In [4]:
# Create single dataframe
df = pd.merge(price, sales, on=["Client", "Warehouse","Product","ds"], how="inner")

df["ds"] = pd.to_datetime(df["ds"])
df = df.astype({"Price": np.float32,
                "y": np.float32,
                "Client": "category",
                "Warehouse": "category",
                "Product": "category",
                })
df

,Client,Warehouse,Product,ds,Price,y
0,0,1,367,2020-07-06,10.900001,7.0
1,0,1,639,2020-07-06,NaN,0.0
2,0,1,655,2020-07-06,21.343332,21.0
3,0,1,1149,2020-07-06,11.480000,7.0
4,0,1,1485,2020-07-06,NaN,0.0
...,...,...,...,...,...,...
2559005,46,318,13485,2023-10-02,26.952000,80.0
2559006,46,318,13582,2023-10-02,24.679487,39.0
2559007,46,318,13691,2023-10-02,52.990002,1.0
2559008,46,318,13946,2023-10-02,30.990000,3.0


In [86]:
df.to_csv("dataset_datathon.csv", index=False)

In [87]:
# Read price data
# price = pd.read_csv("Phase 0 - Price.csv", na_values=np.nan)
# price["Value"] = "Price"

# price = price.set_index(["Client", "Warehouse","Product", "Value"]).stack()
# print(price)
# print()

# # # Read sales data
# sales = pd.read_csv("Phase 0 - Sales.csv", na_values=np.nan)
# sales["Value"] = "Sales"
# sales = sales.set_index(["Client", "Warehouse","Product", "Value"]).stack()
# print()
# print(sales)

# # # Create single dataframe
# df = pd.concat([price, sales]).unstack("Value").reset_index()

# df.columns = ["Client", "Warehouse", "Product", "ds", "Price", "y"]

# df["ds"] = pd.to_datetime(df["ds"])
# df = df.astype({"Price": np.float32,
#                 "y": np.float32,
#                 "Client": "category",
#                 "Warehouse": "category",
#                 "Product": "category",
#                 })
# df = df.sort_values(by="y", ascending=False)
# print(df.head(40))

# 3. Feature Engineering

In [88]:
# Date features
def _get_date_features(df):
    df["Week"] = df["ds"].dt.isocalendar().week
    df["Month"] = df["ds"].dt.month
    df["Year"] = df["ds"].dt.year

    return df


# Target lag transforms
def _get_lag_transforms(lag_transform_weeks = [2, 4, 6, 8]):
    lag_transforms = []
    for lag in lag_transform_weeks:
        lag_transforms.append(RollingMean(window_size=lag))

    return lag_transforms


# Target lags
def _get_lags(lags = [1, 2, 4]):
    return lags



# Price transforms
def _get_price_transforms(df):
    
    # Transform price feature
    transformed = transform_exog(
        df[["unique_id", "ds", "Price"]],
        lags=[1],
    )

    
    # Add deltas
    transformed["Delta_Price_lag1"] = transformed["Price"] - transformed["Price_lag1"]
    transformed["RelDelta_Price_lag1"] = transformed["Price"] / transformed["Price_lag1"] - 1

    
    # Merge
    df_price = transformed.drop(columns=["Price", "Price_lag1"])
    df = df.merge(df_price, left_on = ["unique_id", "ds"], right_on=["unique_id", "ds"])

    return df




# Function required to generate the future exogenous (mainly Price) for the test set
def _get_future_exog(X_df, df):
    X_df_idx = X_df.set_index(["unique_id", "ds"]).index
    df_last_price = df.groupby(["unique_id"])["Price"].last()
    X_df = X_df.merge(df_last_price, left_on=["unique_id"], right_on=["unique_id"])
    X_df[["Client", "Warehouse", "Product"]] = X_df["unique_id"].str.split("/", expand=True)
    df_test = df.copy().merge(X_df, left_on=["unique_id", "ds", "Client", "Warehouse", "Product", "Price"], right_on=["unique_id", "ds", "Client", "Warehouse", "Product", "Price"], how="outer")
    df_test = _get_price_transforms(df_test)
    df_test = _get_date_features(df_test)
    df_test = df_test.drop(columns=["Client", "Warehouse", "Product"])
    df_test = df_test.set_index(["unique_id", "ds"])
    X_df = df_test.loc[X_df_idx]

    price_lag_cols = [col for col in X_df.columns if "Price_lag" in col]
    X_df[price_lag_cols] = X_df[price_lag_cols].fillna(0)

    X_df = X_df.reset_index()
    
    return X_df




# Add unique_id and sort
df["unique_id"] = df["Client"].astype(str) + "/" + df["Warehouse"].astype(str) + "/" + df["Product"].astype(str)
df = df.sort_values(by=["unique_id", "ds"]).reset_index(drop=True)



# There is implied target leakage in the Price feature - where price is unavailable, there is zero sales, and vice versa. So, we first lag the Price feature by one.
# Lag price feature by 1
df_price_lagged = transform_exog(
    df[["unique_id", "ds", "Price"]],
    lags=[1],
)

df = df.merge(df_price_lagged[["unique_id", "ds", "Price_lag1"]], left_on=["unique_id", "ds"], right_on=["unique_id", "ds"])
df = df.drop(columns=["Price"])
df = df.rename(columns={"Price_lag1": "Price"})




# #  We subsequently deal with it very basic: we fill the unknown prices with the mean price of the product from the same client at the same week, and the remaining Price NaNs are forward-filled. 
mean_price_per_product = df.groupby(["Client", "Product", "ds"], observed=True)["Price"].mean().reset_index()

df = df.merge(mean_price_per_product, left_on=["Client", "Product", "ds"], right_on=["Client", "Product", "ds"], suffixes=("", "_mean"))
print(df)
print()
df["Price_mean"] =  df.groupby(["Client", "Product"], observed=True)["Price_mean"].ffill()
print()
print(df)
# df["Price"] = df["Price"].fillna(df["Price_mean"])
# df = df.drop(columns="Price_mean")



# df_price = df[["unique_id", "ds", "Price"]].copy()
# df_price = _get_price_transforms(df_price)
# df_price = df_price.drop(columns = "Price")
# df = df.merge(df_price, left_on = ["unique_id", "ds"], right_on=["unique_id", "ds"])



# # Add date features
# df = _get_date_features(df)

        Client Warehouse Product         ds    y  unique_id      Price  \
0            0         1   10705 2020-07-06  7.0  0/1/10705        NaN   
1            0         1   10705 2020-07-13  7.0  0/1/10705  28.550001   
2            0         1   10705 2020-07-20  7.0  0/1/10705  28.550001   
3            0         1   10705 2020-07-27  7.0  0/1/10705  28.550001   
4            0         1   10705 2020-08-03  7.0  0/1/10705  33.738571   
...        ...       ...     ...        ...  ...        ...        ...   
2559005      9        82    9950 2023-09-04  0.0  9/82/9950        NaN   
2559006      9        82    9950 2023-09-11  0.0  9/82/9950        NaN   
2559007      9        82    9950 2023-09-18  0.0  9/82/9950        NaN   
2559008      9        82    9950 2023-09-25  0.0  9/82/9950        NaN   
2559009      9        82    9950 2023-10-02  0.0  9/82/9950        NaN   

         Price_mean  
0               NaN  
1         16.614653  
2         16.614653  
3         16.614653  
4

In [89]:
df

,Client,Warehouse,Product,ds,y,unique_id,Price,Price_mean
0,0,1,10705,2020-07-06,7.0,0/1/10705,NaN,NaN
1,0,1,10705,2020-07-13,7.0,0/1/10705,28.550001,16.614653
2,0,1,10705,2020-07-20,7.0,0/1/10705,28.550001,16.614653
3,0,1,10705,2020-07-27,7.0,0/1/10705,28.550001,16.614653
4,0,1,10705,2020-08-03,7.0,0/1/10705,33.738571,15.486969
...,...,...,...,...,...,...,...,...
2559005,9,82,9950,2023-09-04,0.0,9/82/9950,NaN,35.000000
2559006,9,82,9950,2023-09-11,0.0,9/82/9950,NaN,35.000000
2559007,9,82,9950,2023-09-18,0.0,9/82/9950,NaN,35.000000
2559008,9,82,9950,2023-09-25,0.0,9/82/9950,NaN,35.000000
